We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

In [51]:
import pandas as pd

df = pd.read_parquet('yellow_tripdata_2023-01.parquet')
len(df.columns)

19

In [52]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Q1: How many columns are there?
 - 19

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [53]:
df.loc[:, 'duration'] = df.loc[:, 'tpep_dropoff_datetime'] - df.loc[:, 'tpep_pickup_datetime']
df.loc[:, 'duration'] = df.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/4186165970.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 8.43333333  6.31666667 12.75       ... 24.51666667 13.
 14.4       ]' has dtype incompatible with timedelta64[us], please explicitly cast to a compatible dtype first.
  df.loc[:, 'duration'] = df.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)


In [54]:
df.duration.std()

42.59435124195458

Q2: What's the standard deviation of the trips duration in January?
- 42.59

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [55]:
# before
print(len(df))
df = df[(df.duration >= 1) & (df.duration <= 60)]
print(len(df))

3066766
3009173


In [56]:
2421440/2463931

0.9827547930522406

Q3: What fraction of the records left after you dropped the outliers?
- 98%

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

In [57]:
categorical = ['PULocationID', 'DOLocationID']
df.loc[:, categorical] = df.loc[:,categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/1679385632.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:,categorical].astype(str)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/1679385632.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, categorical] = df.loc[:,categorical].astype(str)


In [58]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [59]:
X_train.shape

(3009173, 515)

Q4: What's the dimensionality of this matrix (number of columns)?
- 515

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

7.649261927686161

Q5: What's the RMSE on train?
- 7.65

Now let's apply this model to the validation dataset (February 2023).

In [61]:
val = pd.read_parquet('yellow_tripdata_2023-02.parquet')
val.loc[:, 'duration'] = val.loc[:, 'tpep_dropoff_datetime'] - val.loc[:, 'tpep_pickup_datetime']
val.loc[:, 'duration'] = val.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)
val = val[(val.duration >= 1) & (val.duration <= 60)]
val.loc[:, categorical] = val.loc[:,categorical].astype(str)
val_dicts = val[categorical].to_dict(orient='records')

/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/3349567031.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.68333333  0.23333333  0.23333333 ... 14.          7.
  9.8       ]' has dtype incompatible with timedelta64[us], please explicitly cast to a compatible dtype first.
  val.loc[:, 'duration'] = val.loc[:, 'duration'].apply(lambda td: td.total_seconds() / 60)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/3349567031.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  val.loc[:, categorical] = val.loc[:,categorical].astype(str)
/var/folders/dz/nf0874bd6dx7_2jbvw0z7lcw0000gn/T/ipykernel_67380/3349567031.py:5: FutureWarning: Setting an item of incompatible dtype is deprecate

In [62]:
X_val = dv.transform(val_dicts)
y_train = df[target].values
y_val = val[target].values

In [63]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.811817957524739

Q6: What's the RMSE on validation?
- 7.81